# GRCh38 DNase 12 kb CAE Optimizer Search 3

**Tasks:** binary cross-entropy loss leads to reliable training

**Insights:**

- 

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import json
import numpy as np
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
testing = False
    
# The base directory is one level up
base = '..'
name = 'optimizer-search-3'
dataset = 'merged'
settings_filepath = '../settings-grch38-chip-12kb.json'
search_filepath = '../{}.json'.format(name)

with open(settings_filepath, "r") as f:
    settings = json.load(f)

with open(search_filepath, "r") as f:
    search = json.load(f)

## Create training jobs

In [24]:
from jobs import jobs

jobs(
    os.path.relpath(search_filepath, base),
    os.path.relpath(settings_filepath, base),
    dataset=dataset,
    name=name,
    cluster='cox',
    epochs=25,
    batch_size=256,
    repeat=10,
    base=base,
    clear=True,
    verbose=False
)

True



Created slurm file for training 3 neural networks


## Test training run of the most complex CAE

In [ ]:
from train import train_on_single_dataset

with open(os.path.join(base, 'definitions-{}.json'.format(name)), 'r') as f:
    definitions = json.load(f)

train_on_single_dataset(
    settings,
    'merged',
    definitions=definitions,
    definition_idx=1,
    epochs=2,
    batch_size=2048,
    base=base,
    clear=True,
)

In [ ]:
from ae.utils import check_status

okay, not_found, outdated = check_status(
    name, 'training', 'cnn-search', base=base
)

if okay:
    print('Trainings completed')
else:
    print('Trainings did not finish. Missing {}'.format(len(not_found))

## Create evaluation jobs

In [ ]:
from evaluate import create_jobs

create_jobs(
    name,
    name=name,
    dataset=dataset,
    cluster='cox',
    base=base,
    clear=True,
    incl_dtw=False,
)

In a terminal run: `sbatch evaluate-optimizer-search-2.slurm`

In [ ]:
from ae.utils import check_status

okay, not_found, outdated = check_status(
    name, 'evaluate', 'cnn-search', base=base
)

if okay:
    print('Evaluation completed')
else:
    print('Evaluation did not finish. Missing {}'.format(len(not_found))

## Compare

In [ ]:
from compare import compare
    
performance = compare(
    'definitions-{}.json'.format(name),
    dataset_name=dataset,
    base=base,
    clear=False,
    verbose=False,
    silent=False,
    remove_common_prefix_from_df=True,
)

In [ ]:
import qgrid

qgw = qgrid.show_grid(performance)
qgw

In [ ]:
from IPython.core.display import Image, display

for model in qgw.get_selected_df().iterrows():
    print(model[0])
    display(Image(os.path.join(base, 'models', 'cf-{}---predictions-{}.png'.format(model[0], dataset))))